In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

### Extract CSVs into DataFrames

In [3]:
police = "C:/Users/Tony Duron/Project_Two/Resources/police_murders.csv"

police_df = pd.read_csv(police)
police_df.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,2/1/2015,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,2/1/2015,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,3/1/2015,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,4/1/2015,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
4,9,Michael Rodriguez,4/1/2015,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False


In [4]:
race = "C:/Users/Tony Duron/Project_Two/Resources/Resources_shareracebycity.csv"
race_df = pd.read_csv(race)

race_df.head()

,Geographic area,City,share_white,share_black,share_native_american,share_asian,share_hispanic
0,AL,Abanda CDP,67.2,30.2,0,0,1.6
1,AL,Abbeville city,54.4,41.4,0.1,1,3.1
2,AL,Adamsville city,52.3,44.9,0.5,0.3,2.3
3,AL,Addison town,99.1,0.1,0,0.1,0.4
4,AL,Akron town,13.2,86.5,0,0,0.3


### Transform premise DataFrame

In [5]:
new_police_df = police_df[['name', 'manner_of_death', 'gender', 'race', 'city', 'state']].copy()
new_police_df.head()

new_police_df.rename(columns = {'name':'Name','manner_of_death':'Manner_Of_Death', 'gender':'Gender', 'race':'Race','city':'City', 'state':'State'}, inplace = True)
new_police_df.head()

,Name,Manner_Of_Death,Gender,Race,City,State
0,Tim Elliot,shot,M,A,Shelton,WA
1,Lewis Lee Lembke,shot,M,W,Aloha,OR
2,John Paul Quintero,shot and Tasered,M,H,Wichita,KS
3,Matthew Hoffman,shot,M,W,San Francisco,CA
4,Michael Rodriguez,shot,M,H,Evans,CO


### Transform county DataFrame

In [6]:
new_race_df = race_df[['Geographic area', 'City', 'share_white', 'share_black', 'share_native_american', 'share_asian', 'share_hispanic']].copy()
new_race_df.head()

new_race_df.rename(columns = {'Geographic area':'State', 'share_white':'Percent_White', 'share_black':'Percent_Black', 'share_native_american':'Percent_Nat_Amer', 'share_asian':'Percent_Asian', 'share_hispanic':'Percent_Hispanic'}, inplace = True)
new_race_df.head()

,State,City,Percent_White,Percent_Black,Percent_Nat_Amer,Percent_Asian,Percent_Hispanic
0,AL,Abanda CDP,67.2,30.2,0,0,1.6
1,AL,Abbeville city,54.4,41.4,0.1,1,3.1
2,AL,Adamsville city,52.3,44.9,0.5,0.3,2.3
3,AL,Addison town,99.1,0.1,0,0.1,0.4
4,AL,Akron town,13.2,86.5,0,0,0.3


### Create database connection

In [89]:
rds_connection_string = "postgres:postgres@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [90]:
# Confirm tables
engine.table_names()

['premise_file_df',
 'county_file_df',
 'premise_name',
 'county_name',
 'county_name_1',
 'license-name']

### Load DataFrames into database

In [93]:
new_premise_file_df.to_sql(name='new_premise_file', con=engine, if_exists='append', index=False)

In [94]:
new_county_file_df.to_sql(name='new_county_file', con=engine, if_exists='append', index=False)